# FNS-Clean Tungsten benchmark analysis 
*FNS-clean-w* is an intermediate quality neutronics experiment conducted to test the response of thick tungsten to ~14 MeV neutrons [1]. \
Neutronics code results are compared to the experiment measurements. \
Measurements include:
- Flux evaluation via activation foil reaction rate detection
- Neutron energy spectra above 2 MeV measured via spherical NE213 scintillation spectrometer
- Neutron energy spectra between 5 keV and 1 MeV measured via proton recoil gas proportional counters (PRC)
- Gamma energy spectra measured via BC537 liquid organic scintillation
- Nuclear heating measured via thermoluminescent dosimeters (TLD)

Activation foils were located at depths of 0, 7.6, 22.8, 38.0 and 50.7 cm in the shielding mockup.
All spectrometers were located at depths of 7.6, 22.8, and 38.0 cm within the shielding mockup.
TLDs were located at depths of 0, 7.6, 22.8, 38.0 cm within the shielding mockup.

the DT neutron source was located at 20 cm from the shielding mockup (-20 cm from the detector positioned at 0 cm, then).

It is worth to mention that the OpenMC simulation relies on the _weight windows_ variance reduction technique. A local weight windows file (with _wwinp_ extension) has been produced through denovo within the SCALE code [2].

Model, experimental results and MCNP results were provided by the SINBAD repository [3].

The 14 MeV neutron source characteristics of FNS was not provided. FNG neutron source characheristics has been used in substitution.

In [ ]:
import openmc_fusion_benchmarks as ofb
import matplotlib.pyplot as plt
import numpy as np

import helpers

In [ ]:
# read sinbad data
experiment_file = ofb.ResultsFromDatabase('experiment.h5', path='results_database')

In [ ]:
# read openmc results in results_database/
openmc_fendl3_file = ofb.ResultsFromDatabase('openmc-0-14-0_fendl32b.h5', path='results_database')

### Activation foils - reaction rate analysis
The following reaction rates are included:
- $^{93}Nb(n,2n)^{92m}Nb$
- $^{27}Al(n,\alpha)^{24}Na$
- $^{115}In(n,n')^{115m}Co$
- $^{197}Au(n,\gamma)^{198}Au$
- $^{186}W(n,\gamma)^{187}W$

OpenMC evaluation of the reaction rates relies on the **_IRDFF-II_** dosimetry nuclear data library [4]. While the rest of the transport relies on the nuclear data libraries specified in the legend of the plots. A _flux multiplier_ is applied to a _track-length flux_ tally is applied on the foil _cells_.

In [ ]:
# initialize reaction rate plots
reaction_labels = ['$^{93}$Nb(n,2n)$^{92m}$Nb', '$^{27}$Al(n,\u03B1)$^{24}$Na', 
                   '$^{115}$In(n,n\')$^{115m}$In', '$^{197}$Au(n,\u03B3)$^{198}$Au', 
                   '$^{186}$W(n,\u03B3)$^{187}$W']

ylabel = r'Reaction Rate $(\frac{1}{src \cdot atom})$'

In [ ]:
for i,foil in enumerate(helpers.foil_list):

    # extract tallies
    measured = experiment_file.get_tally_dataframe(f'rr_{foil}')
    
    # openmc from results_database/
    openmc_fendl3 = openmc_fendl3_file.get_tally_dataframe(f'rr_{foil}')
    # openmc_endfb8 = openmc_endfb8_file.get_tally_dataframe(f'rr_{foil}')
    
    # plot
    # preprocessing
    dtype_label = reaction_labels[i]
    tickers = np.arange(len(measured))
    xaxis = experiment_file.get_tally_xaxis(f'rr_{foil}')
    means = [measured['mean']]

    # generate figure
    fig, (ax1,ax2) = plt.subplots(nrows=2, ncols=1, figsize=(6, 5), gridspec_kw={'height_ratios': [2, 1.25]}, constrained_layout=True)

    # plot results
    ax1.plot(tickers, measured['mean'], marker='s', ms=10, ls='none', mew=1.5, mec='k', mfc='none', label='Experiment')

    # # openmc results
    ax1.plot(tickers, openmc_fendl3['mean'], marker='*', ms=7, ls='none', color='tab:red', alpha=1, label='openmc-fendl3')
    # ax1.plot(tickers, openmc_fendl3['mean'], marker='.', ms=7, ls='none', color='tab:purple', label='openmc-endfb8')

    ofb.plot_stddev_area(ax=ax2, ticks=tickers, mean=np.ones(len(measured['mean'])), std_dev=measured['std. dev.']/measured['mean'])
    
    # openmc results
    ax2.errorbar(tickers, openmc_fendl3['mean']/measured['mean'], openmc_fendl3['std. dev.']/measured['mean'],
                marker='*', ms=6, capsize=4, barsabove=True, color='tab:red', ls='none', alpha=1, label='_label')
    # ax2.errorbar(tickers, openmc_endfb8['mean']/measured['mean'], openmc_endfb8['std. dev.']/measured['mean'], 
    #             marker='.', ms=6, capsize=4, barsabove=True, color='tab:purple', ls='none', label='_label')

    # adjust
    ofb.add_floor_ceiling(ax=ax1, values=means, scale='log', gap=0.)
    ax1.set_yscale('log')
    ax1.set_ylabel(ylabel, fontsize=12)
    ax1.set_xticks(tickers)
    ax1.tick_params(axis='x', labelbottom=False)
    ax1.tick_params(axis='both', which='both', direction='in')
    ax1.legend(fontsize=12)
    #
    ax2.hlines(1.0, -1, 20, colors='k', linestyles='-',
                linewidth=1, label='_nolegend_')
    ax2.set_xlim([-0.5, tickers[-1] + .6])
    ax2.set_ylim([0.1, 1.75])
    ax2.set_xticks(tickers)
    ax2.set_xticklabels(measured[xaxis])
    ax2.tick_params(axis='x', labelrotation=45)
    ax2.tick_params(axis='both', which='both', direction='in')
    ax2.set_xlabel(xaxis, fontsize=12)
    ax2.set_ylabel('C/E', fontsize=12)
    ax2.annotate(dtype_label, [0.02, 0.07], xycoords='axes fraction',
                        horizontalalignment='left', verticalalignment='bottom', fontsize=12)

### Spectrometers - neutron energy spectra

Neutron energy spectra between 5 keV and 1 MeV measured via PRC and above 2 MeV measured via NE213.

In [ ]:
# initialize spectra data
ylabel = r'Neutron flux $(\frac{n}{cm^2 \cdot src \cdot lethargy})$'

In [ ]:
for i, d in enumerate(helpers.detector_list):
    # extract tallies
    ne213 = experiment_file.get_tally_dataframe(f'nspectrum_ne213_{d}')
    prc = experiment_file.get_tally_dataframe(f'nspectrum_prc_{d}')
    
    # openmc from results_database/
    openmc_fendl3_ne213 = openmc_fendl3_file.get_tally_dataframe(f'nspectrum_ne213_{d}')
    openmc_fendl3_prc = openmc_fendl3_file.get_tally_dataframe(f'nspectrum_prc_{d}')
    # openmc_endfb8 = openmc_endfb8_file.get_tally_dataframe(f'nspectrum_{d}')

    # plot
    # preprocessing
    ne213_mean = ne213['mean'] / np.log(ne213['energy high [eV]']/ne213['energy low [eV]'])
    ne213_stddev = ne213['std. dev.'] / np.log(ne213['energy high [eV]']/ne213['energy low [eV]'])
    prc_mean = prc['mean'] / np.log(prc['energy high [eV]']/prc['energy low [eV]'])
    prc_stddev = prc['std. dev.'] / np.log(prc['energy high [eV]']/prc['energy low [eV]'])

    openmc_fendl3_ne213_mean = openmc_fendl3_ne213['mean'] / np.log(openmc_fendl3_ne213['energy high [eV]']/openmc_fendl3_ne213['energy low [eV]'])
    openmc_fendl3_ne213_stddev = openmc_fendl3_ne213['std. dev.'] / np.log(openmc_fendl3_ne213['energy high [eV]']/openmc_fendl3_ne213['energy low [eV]'])
    openmc_fendl3_prc_mean = openmc_fendl3_prc['mean'] / np.log(openmc_fendl3_prc['energy high [eV]']/openmc_fendl3_prc['energy low [eV]'])
    openmc_fendl3_prc_stddev = openmc_fendl3_prc['std. dev.'] / np.log(openmc_fendl3_prc['energy high [eV]']/openmc_fendl3_prc['energy low [eV]'])
    # openmc_endfb8_mean = openmc_endfb8['mean'] / np.log(openmc_endfb8['energy high [eV]']/openmc_endfb8['energy low [eV]'])
    # openmc_endfb8_stddev = openmc_endfb8['std. dev.'] / np.log(openmc_endfb8['energy high [eV]']/openmc_endfb8['energy low [eV]'])
    
    # generate figure
    fig, (ax1,ax2) = plt.subplots(ncols=2, figsize=(10, 3), constrained_layout=True)

    for ax in (ax1,ax2):
        # plot results
        ax.step(ne213['energy low [eV]'], ne213_mean, lw=1.5, c='k', label='Experiment ne213')
        ax.fill_between(ne213['energy low [eV]'], ne213_mean - ne213_stddev, ne213_mean +
                                ne213_stddev, step='pre', color='k', alpha=0.2)
        ax.step(prc['energy low [eV]'], prc_mean, lw=1.5, ls='--', c='k', label='Experiment prc')
        ax.fill_between(prc['energy low [eV]'], prc_mean - prc_stddev, prc_mean +
                                prc_stddev, step='pre', color='k', alpha=0.2)
    
        # openmc results
        ax.step(openmc_fendl3_ne213['energy low [eV]'], openmc_fendl3_ne213_mean, lw=1.5, c='tab:red', label='openmc-fendl3 ne213')
        ax.fill_between(openmc_fendl3_ne213['energy low [eV]'], openmc_fendl3_ne213_mean - openmc_fendl3_ne213_stddev, 
                        openmc_fendl3_ne213_mean + openmc_fendl3_ne213_stddev, step='pre', color='tab:red', alpha=0.2)
        ax.step(openmc_fendl3_prc['energy low [eV]'], openmc_fendl3_prc_mean, lw=1.5, ls='--', c='tab:red', label='openmc-fendl3 prc')
        ax.fill_between(openmc_fendl3_prc['energy low [eV]'], openmc_fendl3_prc_mean - openmc_fendl3_prc_stddev, 
                        openmc_fendl3_prc_mean + openmc_fendl3_prc_stddev, step='pre', color='tab:red', alpha=0.2)

        # adjust
        ax.set_yscale('log')
        ax.set_xlabel('Energy (eV)', fontsize=12)
        ax.tick_params(axis='both', which='both', direction='in', labelsize=12)

    # adjust
    ax1.set_ylabel(ylabel, fontsize=12)
    ax1.legend(fontsize=12)
    ax2.set_xscale('log')

### Spectrometers - gamma energy spectra

Gamma energy spectra measured via BC537.

In [ ]:
# initialize spectra data
detector_position = ['76', '228', '380']
ylabel = r'Gamma flux $(\frac{p}{cm^2 \cdot src \cdot lethargy})$'

In [ ]:
for i, d in enumerate(helpers.detector_list):
    # extract tallies
    bc537 = experiment_file.get_tally_dataframe(f'gspectrum_bc537_{d}')
    
    # # openmc from results_database/
    openmc_fendl3 = openmc_fendl3_file.get_tally_dataframe(f'gspectrum_bc537_{d}')
    # openmc_endfb8 = openmc_endfb8_file.get_tally_dataframe(f'nspectrum_{d}')

    # plot
    # preprocessing
    bc537_mean = bc537['mean'] / np.log(bc537['energy high [eV]']/bc537['energy low [eV]'])
    bc537_stddev = bc537['std. dev.'] / np.log(bc537['energy high [eV]']/bc537['energy low [eV]'])

    openmc_fendl3_mean = openmc_fendl3['mean'] / np.log(openmc_fendl3['energy high [eV]']/openmc_fendl3['energy low [eV]'])
    openmc_fendl3_stddev = openmc_fendl3['std. dev.'] / np.log(openmc_fendl3['energy high [eV]']/openmc_fendl3['energy low [eV]'])
    # openmc_endfb8_mean = openmc_endfb8['mean'] / np.log(openmc_endfb8['energy high [eV]']/openmc_endfb8['energy low [eV]'])
    # openmc_endfb8_stddev = openmc_endfb8['std. dev.'] / np.log(openmc_endfb8['energy high [eV]']/openmc_endfb8['energy low [eV]'])
    
    # generate figure
    fig, (ax1,ax2) = plt.subplots(ncols=2, figsize=(10, 3), constrained_layout=True)

    for ax in (ax1,ax2):
        # plot results
        ax.step(bc537['energy low [eV]'], bc537_mean, lw=1.5, c='k', label='Experiment bc537')
        ax.fill_between(bc537['energy low [eV]'], bc537_mean - bc537_stddev, bc537_mean +
                                bc537_stddev, step='pre', color='k', alpha=0.2)
    
        # openmc results
        ax.step(openmc_fendl3['energy low [eV]'], openmc_fendl3_mean, lw=1.5, c='tab:red', label='openmc')
        ax.fill_between(openmc_fendl3['energy low [eV]'], openmc_fendl3_mean - openmc_fendl3_stddev, openmc_fendl3_mean +
                                openmc_fendl3_stddev, step='pre', color='tab:red', alpha=0.2)

        # adjust
        ax.set_yscale('log')
        ax.set_xlabel('Energy (eV)', fontsize=12)
        ax.tick_params(axis='both', which='both', direction='in', labelsize=12)

    # adjust
    ax1.set_ylabel(ylabel, fontsize=12)
    ax1.legend(fontsize=12)
    ax2.set_xscale('log')

### Thermoluminescent detectors - nuclear heating analysis
Nuclear heating has been measured via thermoluminescent detectors which measured the dose deposited in _Gy/src_.

For this case, the OpenMC model featuring a _heating tally_ required _photon transport_ activated alongside _neutron transport_. The contributions of _neutrons_, _photons_, _electrons_ and _positrons_ have been evaluated.

In the fng-str benchmark calibration coefficients for the heating tallies were provided [3]. And the deposited dose (Q) was computed as follows:

$$Q = Q_n\cdot c_n\cdot c_e + \sum(Q_i)\cdot c_p  \tag{1}$$

where $Q_n$ is the neutron contribution and $\sum(Q_i)$ is the contribution of the other particles to the nuclear heating \
$C_n$ is the ratio of the TLD/material absorbed neutron dose \
$C_p$ is the ratio of the TLD/material absorbed photon dose \
$C_e$ is the TLD neutron dose efficiency with respect to the photon dose efficiency, taken from published data and weighted over the neutron spectra

The coefficients have not been provided for fns-clean-w and, unlike in the fng-w case, we did not adjust the order of magnitude of $c_n$. We then applied equation (1) with all the coefficients equal to 1. However, it is essential to explore these discrepancies in the approach required for modeling TLD results.

This process is done in the `helpers` and it is automated in the `run_and_store` script.

In [ ]:
ylabel = r'Absorbed Dose $(\frac{Gy}{src})$'
# extract tallies
measured = experiment_file.get_tally_dataframe('nuclear_heating')

# # openmc from results_database
openmc_fendl3 = openmc_fendl3_file.get_tally_dataframe('nuclear_heating')
# openmc_endfb8 = openmc_endfb8_file.get_tally_dataframe('nuclear_heating')

# plot
# preprocessing
dtype_label = reaction_labels[i]
tickers = np.arange(len(measured))
xaxis = experiment_file.get_tally_xaxis('nuclear_heating')
means = [measured['mean']]

# generate figure
fig, (ax1,ax2) = plt.subplots(nrows=2, ncols=1, figsize=(6, 5), gridspec_kw={'height_ratios': [2, 1.25]}, constrained_layout=True)

# plot results
ax1.plot(tickers, measured['mean'], marker='s', ms=10, ls='none', mew=1.5, mec='k', mfc='none', label='Experiment')

# openmc results
ax1.plot(tickers, openmc_fendl3['mean'], marker='*', ms=7, ls='none', color='tab:red', alpha=1, label='openmc-fendl3')
# ax1.plot(tickers, openmc_fendl3['mean'], marker='.', ms=7, ls='none', color='tab:purple', label='openmc-endfb8')

ofb.plot_stddev_area(ax=ax2, ticks=tickers, mean=np.ones(len(measured['mean'])), std_dev=measured['std. dev.']/measured['mean'])
    
# openmc results
ax2.errorbar(tickers, openmc_fendl3['mean']/measured['mean'], openmc_fendl3['std. dev.']/measured['mean'], 
        marker='*', ms=6, capsize=4, barsabove=True, color='tab:red', ls='none', alpha=1, label='_label')
# ax2.errorbar(tickers, openmc_endfb8['mean']/measured['mean'], openmc_endfb8['std. dev.']/measured['mean'], 
#                 marker='.', ms=6, capsize=4, barsabove=True, color='tab:purple', ls='none', label='_label')

# adjust
ofb.add_floor_ceiling(ax=ax1, values=means, scale='log', gap=0.)
ax1.set_yscale('log')
ax1.set_ylabel(ylabel, fontsize=12)
ax1.set_xticks(tickers)
ax1.tick_params(axis='x', labelbottom=False)
ax1.tick_params(axis='both', which='both', direction='in')
ax1.legend(fontsize=12)
#
ax2.hlines(1.0, -1, 20, colors='k', linestyles='-',
            linewidth=1, label='_nolegend_')
ax2.set_xlim([-0.5, tickers[-1] + .6])
ax2.set_ylim([0.5, 1.5])
ax2.set_xticks(tickers)
ax2.set_xticklabels(measured[xaxis])
ax2.tick_params(axis='x', labelrotation=45)
ax2.tick_params(axis='both', which='both', direction='in')
ax2.set_xlabel(xaxis, fontsize=12)
ax2.set_ylabel('C/E', fontsize=12)

plt.show()

## References
[1] Maekawa, F., et al. "Data collection of fusion neutronics benchmark experiment conducted at FNS/JAERI". No. JAERI-DATA/CODE--98-021. Japan Atomic Energy Research Inst., 1998. https://inis.iaea.org/search/search.aspx?orig_q=RN:29054903 \
[2] Evans, Thomas M., et al. "Denovo: A new three-dimensional parallel discrete ordinates code in SCALE." Nuclear technology 171.2 (2010): 171-200. https://doi.org/10.13182/NT171-171 \
[3] Kodeli, I. A., et al. "SINBAD–Radiation shielding benchmark experiments." Annals of Nuclear Energy 159 (2021): 108254. https://doi.org/10.1016/j.anucene.2021.108254 \
[4] Trkov, A., et al. "IRDFF-II: a new neutron metrology library." Nuclear Data Sheets 163 (2020): 1-108. https://doi.org/10.1016/j.nds.2019.12.001